In [1]:
import numpy as np
import pygame
import sys

from quaternion import Point3D, Rotation, Vecteur3D

from drawing_obj import Point_obj, Groupe_drawable_object, Screen, Vecteur_obj, Droite_obj, Circle_obj

%load_ext autoreload
%autoreload 2

pygame 2.6.1 (SDL 2.28.4, Python 3.12.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
points = [
    Point_obj(x, y, z)
    for x in (1.,3.) for y in (-1.,1.) for z in (-1.,1.)
]

cube_point = points[0]
for p in points[1:]:
    cube_point = cube_point | p

d1 = Droite_obj( Point3D([3],[1],[1]), Point3D([1],[1],[1,]) )
# d2 = Droite_obj( Point3D([3],[-1],[1]), Point3D([1],[-1],[1,]) )

# dd = d1 | d1 | d1

# d = Droite_obj( Point3D([3,3,3,3],[1,-1.,1,-1],[1,1,-1,-1]), Point3D([1,1,1,1],[1,-1,1,-1],[1,1,-1,-1]) )
# d = d | Droite_obj( Point3D([1,3,3,1],[-1,-1,-1,-1],[1,1,-1,-1]), Point3D([1,3,3,1],[1,1,1,1],[1,1,-1,-1]) )


for count, i in enumerate(np.linspace( -40,40, 21 )):
    if count:
        grille = grille | Droite_obj( Point3D(1,i,0.), Point3D(0.,i,0.) )
        grille = grille | Droite_obj( Point3D(i,1.,0.), Point3D(i,0.,0.) )
    else:
        grille = Droite_obj( Point3D(1,i,0.), Point3D(0.,i,0.) )
        grille = grille | Droite_obj( Point3D(i,1.,0.), Point3D(i,0.,0.) )

#grille = Droite_obj( Point3D(1,16,0.), Point3D(0.,16,0.) )
#grille = grille | Droite_obj( Point3D(1,-16,0.), Point3D(0.,-16,0.) )

# grille = [ Droite_obj( Point3D(1,i,0.), Point3D(0.,i,0.) ) for i in np.linspace( -3,3, 7 )]
# grille += [ Droite_obj( Point3D(i,1.,0.), Point3D(i,0.,0.) ) for i in np.linspace( -3,3, 11 )]

# vs  = [ Vecteur3D( Point3D(0.,i,0.), Point3D(1,i,0.) ) for i in np.linspace( -3,3, 7 )]
# vs += [ Vecteur3D( Point3D(i,0.,0.), Point3D(i,1.,0.) ) for i in np.linspace( -3,3, 7 )]

vss = Vecteur_obj( Point3D(0.,0.,0.), Point3D(1,0,0.) )
for i in np.linspace( -3,3, 7 ):
    vss = vss | Vecteur_obj( Point3D(0.,i,0.), Point3D(1,i,0.) )
    vss = vss | Vecteur_obj( Point3D(i,0.,0.), Point3D(i,1.,0.) )

#vss = Vecteur3D( Point3D(0.,0.,0.), Point3D(3,0,0.) )


C = Circle_obj( Vecteur3D(Point3D(1,0,0), Point3D(0,0,0)).to_unitaire, 5, 20  )

# rond = Sphere_obj( Point3D(1,1,1), 1., 128 )

grp = Groupe_drawable_object()
grp.add_in( cube_point )
#grp.add_in( d1 )
grp.add_in( grille )
#grp.add_in( vss )
#grp.add_in(rond)
grp.add_in(C)

In [ ]:
SIZE, FPS = 600, 30
S = Screen(2., 110, size_pixel=SIZE, DoV=10.)

pygame.init()
screen = pygame.display.set_mode((SIZE, SIZE))
clock = pygame.time.Clock()

alpha_speed_translation = 0.1
alpha_speed_rot = 3
d_angle = np.arctan( S.demi_screen_size / 1.5 ) / (SIZE/2)  * alpha_speed_rot ### rad/px
rot_y = Rotation(d_angle, Vecteur3D( Point3D(2.,0.,1.), Point3D(2.,0.,0.)) )
rot_z = Rotation(d_angle, Vecteur3D( Point3D(2.,1.,0.), Point3D(2.,0.,0.)) )

rot_y_view = Rotation(d_angle, Vecteur3D( Point3D(0.,0.,1.) ) )
rot_z_view = Rotation(d_angle, Vecteur3D( Point3D(0.,1.,0.) ) )

### 
#rot_cercle = Rotation(d_angle, C.vecteur_normal )
rot_cercle = Rotation(d_angle, C.vecteur_normal.copy )
C.add_rot( rot_cercle )

grp.add_in( Vecteur_obj(rot_cercle.vect_dir.pa, rot_cercle.vect_dir.po) )
grp.add_in( Vecteur_obj(C.vecteur_normal.pa, C.vecteur_normal.po) )

# Main loop
running = True
dragging = False
dyr, dzr = 0, 0
while running:
    for e in pygame.event.get():
        if e.type == pygame.QUIT:
            running = False

        ### Translations
        keys = pygame.key.get_pressed()
        if keys[pygame.K_z]:
            grp >>= (Vecteur3D( Point3D(-1.,0.,0.) ) * alpha_speed_translation)
        if keys[pygame.K_s]:
            grp >>= (Vecteur3D( Point3D(1.,0.,0.) ) * alpha_speed_translation)
        if keys[pygame.K_q]:
            grp >>= (Vecteur3D( Point3D(0.,-1.,0.) ) * alpha_speed_translation)
        if keys[pygame.K_d]:
            grp >>= (Vecteur3D( Point3D(0.,1.,0.) ) * alpha_speed_translation)
        if keys[pygame.K_SPACE]:
            grp >>= (Vecteur3D( Point3D(0.,0.,-1.) ) * alpha_speed_translation)
        if keys[pygame.K_LCTRL]:
            grp >>= (Vecteur3D( Point3D(0.,0.,1.) ) * alpha_speed_translation)

        left, middle, right = pygame.mouse.get_pressed()
        ### rotations
        if right and e.type == pygame.MOUSEMOTION:
            dyr, dzr = e.rel
            rot_y_view.angle = d_angle * dyr
            grp *= rot_y_view
            rot_z_view.angle = d_angle * (-dzr)
            grp *= rot_z_view
        elif left and e.type == pygame.MOUSEMOTION:
            dyr, dzr = e.rel
            rot_y.angle = d_angle * dyr
            grp *= rot_y
            rot_z.angle = d_angle * (-dzr)
            grp *= rot_z

    print("before : ", rot_cercle.vect_dir.v.norm )
    grp.apply_rotation_propre()
    print( rot_cercle.vect_dir.v.norm )

            
    screen.fill((20,20,20))

    ### Draw ojects on screen
    for obj in grp.list_of_obj:

        i, j = obj.project_on_screen(S)

        if i.ndim==1:
            for ii, jj in zip(i,j):
                pygame.draw.circle(screen, (250,250,250), (jj, ii), 6)

        elif i.ndim==2:
            for ii, jj in zip( i.T, j.T ): 
                pygame.draw.line(screen, (250,250,250), (jj[0],ii[0]), (jj[1],ii[1]), 2)

        else:
            print( f"Drawing of obj {type(obj)} not inplemented" )

    pygame.display.flip()
    dt = clock.tick(FPS)
    if dt>=40:
        print( f"ping : {dt} ms" )

pygame.quit()
sys.exit()

[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]


/home/nicolas-gillet/Projets/quaternion/drawing_obj.py:296: RuntimeWarning: invalid value encountered in sqrt
  tpos = np.sqrt(
/home/nicolas-gillet/Projets/quaternion/drawing_obj.py:373: RuntimeWarning: divide by zero encountered in divide
  return (-f_ * ca + f_ * cs - cs * xa_) / (cs * Vx_ + f_ * Vc_)
/home/nicolas-gillet/Projets/quaternion/drawing_obj.py:366: RuntimeWarning: invalid value encountered in multiply
  return -f_ * (Vc_ * ts + ca) / (-f_ + Vx_ * ts + xa_)
/home/nicolas-gillet/Projets/quaternion/drawing_obj.py:366: RuntimeWarning: invalid value encountered in divide
  return -f_ * (Vc_ * ts + ca) / (-f_ + Vx_ * ts + xa_)
/home/nicolas-gillet/Projets/quaternion/drawing_obj.py:380: RuntimeWarning: invalid value encountered in multiply
  x3d1 = Ax + Vx * t1
/home/nicolas-gillet/Projets/quaternion/drawing_obj.py:381: RuntimeWarning: invalid value encountered in multiply
  y3d1 = Ay + Vy * t1
/home/nicolas-gillet/Projets/quaternion/drawing_obj.py:382: RuntimeWarning: invalid 

[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.0235159]
[1.15797495]
[1.28971621]
[1.38584013]
[1.40740514]
[1.42221948]
[1.42221948]
[1.42221948]
[1.42221948]
[1.42221948]
[1.42221948]
[1.42221948]
[1.42221948]
[1.42221948]
[1.42221948]
[1.42221948]
[1.42221948]
[1.42221948]
[1.42221948]
[1.42221948]
[1.42221948]
[1.42221948]
[1.42221948]
[1.42221948]
[1.42221948]
[1.52855737]
[1.62277195]
[1.73320728]
[1.8342263]
[1.91730835]
[2.0114595]
[2.08268092]
[2.0911307]
[2.09383761]
[2.10144313]
[2.10112394]
[2.09574925]
[2.09865682]
[2.09787462]
[2.09888942]
[2.1035884]
[2.12148883]
[2.186953]
[2.28257499]
[2.291672]
[2.29938641]
[2.33612262]
[2.3684402]
[2.41238931]
[2.48301485]
[2.53565065]
[2.55900593]
[2.58047143]
[2.59796276]
[2.59609091]
[2.5731998]
[2.56254114]
[2.48991012]
[2.41959536]
[2.32490155]
[2.19812266]
[2.06886933]
[1.99176499]
[1.92064629]
[1.9154601]
[1.92940242]
[1.945163

SystemExit: 

/home/nicolas-gillet/micromamba/envs/generale/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3675: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
rot_cercle.vect_dir.v.norm

array([2.45101568])